In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf 
import math as m


# more libraries required
from sklearn.preprocessing import OneHotEncoder, normalize, MinMaxScaler

In [2]:
'''Function for preparing dataframes and creating features: pol_count (how many policies id has), set of policies (set of names of the policies the id has) and no unique products (no of policy types the id has). 
creating dataframe with only numerical columns (floats and ints data types) and aggregating the information for each id. 
For values that are the same over multiple records (eg no. of dependents) for a id, keep value as is and for values that differ (eg premiums) sum the values.
 creating a dataframe of the categorical columns (objects, boolean and categories data types), hot-one encode this dataframe and aggregating per id by summing the rows then replacing values >1 with one
   (eg for id that appears twice, gender might have value 2, replace 2 with 1 for flagging gender). Merge the categorical and numerical dataframes to create an aggregated dataframe of all data.
     
'''
def df_prep(df):
    pol_col = df.columns[-1] #name of policy column
    id_col = df.columns[0] #name of id column

     # Count the number of policies per user
    df['pol_count'] = 1
    temp_df = df.groupby(id_col)['pol_count'].sum().reset_index()

    # Merge policy count back into the original DataFrame
    df = df.drop(columns={'pol_count'})
    df = pd.merge(df, temp_df, 
                         on=id_col, 
                         how='left')
    
    # Create a set of unique policies per user
    temp_df = pd.DataFrame(df.groupby(id_col)[pol_col].apply(set))
    temp_df = temp_df.rename(columns={pol_col: 'set of policies'})  #column containing a set of all policy types client has
    df = pd.merge(df, temp_df,on = id_col, how = 'left')

    # Count unique policy types per user
    df['no unique products'] = df['set of policies'].apply(lambda x: len(x) if isinstance(x, set) else 0)

    # Process numerical features
    num_df = df.iloc[:,1:].select_dtypes(include=['float64','int64']).copy() #creating numerical features df
    num_df.insert(0,id_col,df[id_col])
    num_df.iloc[:,1:] = num_df.iloc[:,1:].fillna(0.0)
    
    def custom_agg(series): #if the numerical value is the same accross all rows per client don't sum, take value (eg, how many policies), otherwise sum (eg. premium for policy)
        return series.iloc[0] if len(series.unique()) == 1 else series.sum()

    # Group by 'id' and apply the aggregation
    num_df = num_df.groupby(id_col).agg({col: custom_agg for col in num_df.columns[1:]}).reset_index()

    #creating categorical feature df
    cat_df = df.iloc[:,1:-2].select_dtypes(include=['object','bool','category']).copy()
    
    cat_df.insert(0,id_col,df[id_col])
    cat_df = pd.get_dummies(cat_df.iloc[:,1:],columns = cat_df.iloc[:,1:].columns, dtype=int)
    cat_df.insert(0,id_col,df[id_col])
    cat_df = cat_df.groupby(id_col).sum()
    cat_df = cat_df[cat_df.columns].map(lambda x: 1 if x > 0 else 0)
    
    df = df[[id_col, 'set of policies']]
    # Convert set columns to sorted tuples so they're hashable and comparable
    df['set of policies'] = df['set of policies'].apply(
    lambda x: tuple(sorted(x)) if isinstance(x, set) else tuple())
    df = df.drop_duplicates()
    df['set of policies'] = df['set of policies'].apply(set)

    #merging categorical and numerical aggregated data
    agg_df = pd.merge(cat_df, num_df,on = id_col, how = 'inner') #aggregated dataframe
    cat_col = len(cat_df.columns) #how many categorical columns there are
    pol_columns = [col for col in agg_df.columns if col.startswith(pol_col)] #list of all policy columns
    
    agg_df = pd.merge(agg_df, df,on = id_col, how = 'inner')
   
    return agg_df, pol_columns, cat_col, id_col, pol_col

''' 
 agg_df is the complete aggregated data
 cat_col is the number of categorical columns, to be used in another function
 id_col is the name of the id column
 pol_col is the name of the policy column '''

' \n agg_df is the complete aggregated data\n cat_col is the number of categorical columns, to be used in another function\n id_col is the name of the id column\n pol_col is the name of the policy column '

Getting the training data with its recommendations

In [ ]:
file_path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/Rot3 AA/Recommender phase 2/datasets/backtesting/carnpar_202404_recs_v4.1_4k.csv"
Rx =  pd.read_csv(file_path) #output from recommender after running it on df_x 
pids = Rx[id_col].drop_duplicates() #pids that have recommendations, training pids

Rx['no.1 most rec product'] = Rx['no.1 most rec product'].str.replace('pol_type_desc_', '') #cleaning up
Rx['no.2 most rec product'] = Rx['no.2 most rec product'].str.replace('pol_type_desc_', '')
Rx['no.3 most rec product'] = Rx['no.3 most rec product'].str.replace('pol_type_desc_', '')
Rx['no.4 most rec product'] = Rx['no.4 most rec product'].str.replace('pol_type_desc_', '')
Rx['no.5 most rec product'] = Rx['no.5 most rec product'].str.replace('pol_type_desc_', '')
Rx['no.6 most rec product'] = Rx['no.6 most rec product'].str.replace('pol_type_desc_', '')


Getting current data

In [ ]:
file_path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/Rot3 AA/Recommender phase 2/datasets/backtesting/carnpar_current"
df_c =  pd.read_csv(file_path) #up-to-date data
df_c = df_c[[col for col in df_c.columns if col != 'pol_type_desc'] + ['pol_type_desc']]
print('no. clients',len(df_c.groupby([id_col])[id_col].count()))
Dc, pol_columns_c, cat_col_c, id_col, pol_col_c = df_prep(df_c)
Dc

no. clients 315034


,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST TRUSTS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,pol_count,no unique products,set of policies
0,84,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,720.0,3,2,"{IFN - Personal Portfolio, Retirement Annuitie..."
1,88,0,0,0,0,0,0,0,1,0,...,0.0,3.0,1.0,0.0,1.0,5.0,0.0,7,6,"{Endowment, Retirement Annuities : PPS, Top Co..."
2,97,0,1,0,1,0,0,1,0,0,...,0.0,3.0,1.0,0.0,0.0,3.0,659.0,11,3,"{IFN - Personal Portfolio, SANTAM - Short Term..."
3,3047,0,1,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,3,3,"{IFN - Personal Portfolio, Retirement Annuitie..."
4,3381,0,1,0,0,0,0,1,1,0,...,0.0,0.0,0.0,1.0,1.0,4.0,714.0,5,5,"{IFN - Personal Portfolio, SPF - Corporate Inv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315029,14102740,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8,1,{SKY - Funeral Policies}
315030,14102866,0,0,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,3,"{SPF - Corporate Investment, Term Annuities, G..."
315031,14115353,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,598.0,1,1,{Retirement Annuities :Non PPS}
315032,14121144,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,2,"{SPF - Trust Wills, SANLAM - Personal Loan}"


Get current data for training data

In [ ]:
Dc_s = pd.merge(Dc,pids,on=id_col,how='inner') #getting current info for the training pids
Dc_s

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST TRUSTS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,pol_count,no unique products,set of policies
0,3047,0,1,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,3,3,"{IFN - Personal Portfolio, Retirement Annuitie..."
1,24203,0,1,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,1.0,3.0,687.0,7,3,"{IFN - Personal Portfolio, SPF - Corporate Inv..."
2,33409,0,0,0,0,0,0,0,1,0,...,1.0,1.0,1.0,1.0,0.0,1.0,726.0,1,1,{Retirement Annuities :Non PPS}
3,44298,0,0,0,0,0,0,1,1,0,...,0.0,1.0,1.0,1.0,1.0,6.0,680.0,5,5,"{Endowment, SPF - Corporate Investment, Retire..."
4,70889,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,2,"{IFN - Personal Portfolio, SPF - Corporate Inv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9784,12752217,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,579.0,5,1,{SKY - Funeral Policies}
9785,12753157,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,2.0,665.0,2,2,"{Retirement Annuities :Non PPS, Top Cover}"
9786,12755452,0,1,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,644.0,3,1,{IFN - Personal Portfolio}
9787,13457624,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,604.0,2,1,{SKY - Funeral Policies}


Getting the data up to 202404 (of which a portion is trained on)

In [6]:
file_path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/Rot3 AA/Recommender phase 2/datasets/backtesting/carnpar_202404"
df_x =  pd.read_csv(file_path) 
df_x = df_x[[col for col in df_x.columns if col != 'pol_type_desc'] + ['pol_type_desc']]
Dx, pol_columns_x, cat_col_x, id_col, pol_col_x = df_prep(df_x)
display(Dx.head())

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST WILLS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,pol_count,no unique products,set of policies
0,84,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,720.0,2,2,"{IFN - Personal Portfolio, Retirement Annuitie..."
1,88,0,0,0,0,0,0,0,1,1,...,0.0,3.0,1.0,0.0,1.0,5.0,0.0,6,6,"{Endowment, Retirement Annuities : PPS, Top Co..."
2,97,0,1,0,1,0,0,1,0,0,...,0.0,3.0,1.0,0.0,0.0,3.0,659.0,6,3,"{IFN - Personal Portfolio, SANTAM - Short Term..."
3,3047,0,1,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,3,3,"{IFN - Personal Portfolio, Retirement Annuitie..."
4,3381,0,1,0,0,0,0,1,1,0,...,0.0,0.0,0.0,1.0,1.0,4.0,714.0,5,5,"{IFN - Personal Portfolio, SPF - Corporate Inv..."


Get the back-dated data for the training data

In [ ]:
Dx_s = pd.merge(Dx,pids,on=id_col,how='inner') #getting current info for the training pids
Dx_s

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST WILLS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,pol_count,no unique products,set of policies
0,3047,0,1,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,3,3,"{IFN - Personal Portfolio, Retirement Annuitie..."
1,24203,0,1,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,1.0,3.0,687.0,5,3,"{IFN - Personal Portfolio, SPF - Corporate Inv..."
2,33409,0,0,0,0,0,0,0,1,0,...,1.0,1.0,1.0,1.0,0.0,1.0,726.0,1,1,{Retirement Annuities :Non PPS}
3,44298,0,0,0,0,0,0,0,1,1,...,0.0,1.0,1.0,1.0,1.0,6.0,680.0,4,4,"{Retirement Annuities :Non PPS, SPF - Trust Wi..."
4,70889,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2,2,"{IFN - Personal Portfolio, SPF - Corporate Inv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9784,12752217,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,579.0,2,1,{SKY - Funeral Policies}
9785,12753157,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,2.0,665.0,1,1,{Top Cover}
9786,12755452,0,1,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,644.0,1,1,{IFN - Personal Portfolio}
9787,13457624,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,604.0,1,1,{SKY - Funeral Policies}


Joining the current and old (202404) data

In [8]:
df_j = pd.merge(Dx_s,Dc_s[[id_col,'no unique products','set of policies','pol_count']],
                on = id_col,
                how = 'inner', suffixes=('_x', '_c'))

#flag whether client has new policy and flag for if client has new product (bc client can take ut new policy of product they already have)
df_j['new_policy'] = (df_j['pol_count_x'] < df_j['pol_count_c']).astype(int)
df_j['new_product'] = (df_j['no unique products_x'] < df_j['no unique products_c']).astype(int)
#getting the name of the new product
df_j['newest_product'] = [c-x if f == 1 else 0 
                          for x,c,f in zip(df_j['set of policies_x'],df_j['set of policies_c'],df_j['new_product'])]
df_j.head()

df_new = df_j[df_j['newest_product'] != 0] #dataframe of users with new products
df_new = pd.merge(df_new,Rx[[id_col,'no.1 most rec product','no.1 product score','no.2 most rec product','no.2 product score','no.3 most rec product','no.3 product score','no.4 most rec product','no.4 product score','no.5 most rec product','no.5 product score','no.6 most rec product','no.6 product score']],
                  on = id_col, 
                  how='inner') #include recommendations


In [ ]:
df_new.iloc[:,-17:]

In [9]:
#scoring based on where new product sits in regards to recommendations
#MRR_4 score
df_new['MRR_4_score'] = [
    1 if f_rec in s else 0.75 if s_rec in s else 0.5 if t_rec in s else 0.25 if fr_rec in s else 0
    for f_rec,s_rec,t_rec,fr_rec,s in zip(df_new['no.1 most rec product'],df_new['no.2 most rec product'],df_new['no.3 most rec product'],df_new['no.4 most rec product'],df_new['newest_product'])
]
print(np.mean(df_new['MRR_4_score']))

0.2823798172427493


In [10]:
pd.DataFrame(df_new['MRR_4_score']).groupby(['MRR_4_score'])['MRR_4_score'].count()

MRR_4_score
0.00    1471
0.25     191
0.50     237
0.75     294
1.00     324
Name: MRR_4_score, dtype: int64

In [11]:
'''Function to calculate the mean reciprical rank score using "showntell" dataframes'''

def MRR(df,pol_col): #mean reciprical rank_3
 
    conditions = [df.apply(lambda row: len({row['no.1 most rec product']}.intersection(row[pol_col])) ,axis=1) == 1,
                  df.apply(lambda row: len({row['no.2 most rec product']}.intersection(row[pol_col])) ,axis=1) == 1, 
                  df.apply(lambda row: len({row['no.3 most rec product']}.intersection(row[pol_col])) ,axis=1) ==1]
    scores = [1, 0.66, 0.33]
    df['MRR_3_score'] = np.select(conditions, scores, default=0)
    return df

'''df is the dataframe given with MRR scores appended'''

'df is the dataframe given with MRR scores appended'

In [12]:
df_new = MRR(df_new,'newest_product')
df_new['MRR_3_score'].describe()

count    2517.000000
mean        0.236889
std         0.365781
min         0.000000
25%         0.000000
50%         0.000000
75%         0.330000
max         1.000000
Name: MRR_3_score, dtype: float64

In [13]:
display(df_new)

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST WILLS,...,no.3 most rec product,no.3 product score,no.4 most rec product,no.4 product score,no.5 most rec product,no.5 product score,no.6 most rec product,no.6 product score,MRR_4_score,MRR_3_score
0,44298,0,0,0,0,0,0,0,1,1,...,Committed Plan,0.924089,Top Cover,0.917481,SANLAM - Personal Loan,0.913389,Child Insurance,0.911548,0.00,0.00
1,144805,0,0,0,0,0,0,0,1,1,...,Whole Life,0.920226,SANTAM - Short Term Insurance,0.895062,Child Insurance,0.892439,Top Cover,0.888593,0.00,0.00
2,186635,0,1,0,0,0,0,1,0,0,...,Whole Life,0.866370,Top Cover,0.858277,SANTAM - Short Term Insurance,0.853565,Child Insurance,0.850701,0.00,0.00
3,188582,0,0,0,1,0,0,1,1,0,...,Top Cover,0.901425,Child Insurance,0.897696,SANLAM - Personal Loan,0.894609,Achievement Award,0.890793,0.50,0.33
4,197907,0,0,0,0,0,0,0,1,0,...,Term Annuities,0.927063,Flexible Plan,0.906871,Endowment Continuation,0.899051,PreservationFund,0.877491,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,12716121,0,0,0,0,0,0,0,1,0,...,SANTAM - Short Term Insurance,0.909725,Top Cover,0.904636,Whole Life,0.903189,Committed Plan,0.892862,1.00,1.00
2513,12722040,0,0,0,0,0,0,0,1,0,...,Top Cover,0.928890,Child Insurance,0.927994,Achievement Award,0.920829,SANLAM - Personal Loan,0.916665,0.50,0.33
2514,12724884,0,0,0,0,0,0,1,0,1,...,IFN - Personal Portfolio,0.923435,Top Cover,0.910928,SANLAM - Personal Loan,0.909665,Child Insurance,0.909164,0.25,0.00
2515,12726348,0,0,0,0,0,0,0,1,0,...,Achievement Award,0.919045,SANTAM - Short Term Insurance,0.917042,Top Cover,0.916734,Child Insurance,0.912885,0.00,0.00
